<a href="https://colab.research.google.com/github/OlegV12/NaturalLangProcessing/blob/Lesson_13/HW_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [58]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from tqdm import tqdm
from collections import Counter
from scipy import spatial


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import transformers
from transformers import pipeline

In [4]:
pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 880 kB 11.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=0dc9ab72e10b25986fdd6c9568e1e5d95a982f006b722d8e53a3ff21ac38639f
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [12]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 12.8 MB/s 
     |████████████████████████████████| 115 kB 61.5 MB/s 
     |████████████████████████████████| 141 kB 59.4 MB/s 
     |████████████████████████████████| 212 kB 61.3 MB/s 
     |████████████████████████████████| 127 kB 47.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [13]:
from datasets import load_dataset

corpus = load_dataset('merionum/ru_paraphraser')



Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-aad01005423eb70c/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [48]:
from transformers import AutoTokenizer, AutoModel
import torch

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


tokenizer = AutoTokenizer.from_pretrained('symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli')
model = AutoModel.from_pretrained('symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli')





In [49]:
def get_cosine(text_1, text_2):
  encoded_input = tokenizer([text_1, text_2], padding=True, truncation=True, return_tensors='pt')
  with torch.no_grad():
    model_output = model(**encoded_input)
  sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
  return spatial.distance.cosine(sentence_embeddings[0], sentence_embeddings[1])


In [53]:
get_cosine(corpus['train'][0]['text_1'], corpus['train'][0]['text_2'])

0.21523058414459229

In [60]:
corpus['train']

Dataset({
    features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
    num_rows: 7227
})

In [62]:
df = pd.DataFrame(corpus['train'])

In [67]:
df['predict'] = df.apply(lambda x: get_cosine(x['text_1'], x['text_2']), axis=1)

In [87]:
df.head()

,id,id_1,id_2,text_1,text_2,class,predict
0,1,201,8159,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,0.215231
1,2,202,8158,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,0.357404
2,3,273,8167,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0,0.223787
3,4,220,8160,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1,0.517637
4,5,223,8160,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0,0.406873


In [89]:
df.loc[df['class']=='1']['predict'].median()

0.1828460395336151

In [90]:
df.loc[df['class']=='0']['predict'].median()

0.3248597979545593

In [91]:
df.loc[df['class']=='-1']['predict'].median()

0.7087665349245071